# US School Analysis

## Read in Data

In [ ]:
# Include all your imports here
import pandas as pd
from pathlib import Path
import seaborn as sns


In [ ]:
# Load the Public school dataset
# Paths
school_data_path = Path("Resources/public_school_data.csv")
school_dict_path = Path("Resources/school_data_dictionary.csv")
locations_path = Path("Resources/public_school_locations.csv")

# Dataframes
raw_data_df = pd.read_csv(school_data_path)
locations_df = pd.read_csv(locations_path)
school_dict = pd.read_csv(school_dict_path)

## Data Exploration and Preparation

In [ ]:
# Keep only fields that appear in the main data
dictionary = {}
valid_fields = school_dict["Fields"].isin(raw_data_df.columns) | school_dict["Fields"].isin(locations_df.columns)
selected_rows = school_dict[valid_fields]

for index, row in selected_rows.iterrows():
    dictionary[row["Fields"]] = row["Field Descriptions"]
    
dictionary


In [ ]:
# Ensure the column you selected is indeed a unique identifier by checking the number of uniqe values and comparing it to the number of rows in the dataframe
if len(raw_data_df["NCESSCH"].value_counts()) == raw_data_df.shape[0]:
    print("NCESSCH IS a unique identifier.")
else:
    print("NCESSCH is NOT a unique identifier.")


In [ ]:
# Change the names of the unique ID column in both the location data and public school data to match and set them as index
def rename_columns(df, dictionary):
    for old_column, new_column in dictionary.items():
        if old_column in df.columns:
            df.rename(columns={old_column: new_column}, inplace=True)
        else:
            print(f"Column '{old_column}' not found in {df}.")
            
    return df
    
school_data = rename_columns(raw_data_df, dictionary)
school_data.set_index("Unique School ID", inplace = True)

location_data = rename_columns(locations_df, dictionary)
location_data.rename(columns={"ID":"Unique School ID"}, inplace=True)
location_data.set_index("Unique School ID", inplace = True)




In [ ]:
# Confirm
school_data.head()

In [ ]:
# Confirm
location_data.head()

In [ ]:
# Join location and raw data 
all_data = school_data.join(location_data, on="Unique School ID", how="left")
all_data.head()

In [ ]:
# Get the value counts for the school type column
all_data["School type (description)"].value_counts()

In [ ]:
# Create a subset of the dataframe that only includes regular schools 
regular_schools = all_data.loc[all_data["School type (description)"] == "Regular school"]
if len(regular_schools) == all_data["School type (description)"].value_counts()["Regular school"]:
    print("Regular school data collected succesfully")
else:
    print("Error creating Regular school dataframe")

## Data Cleaning and dealing with nulls

In [ ]:
# Check for nulls
print(regular_schools.isnull().sum())


In [ ]:
# Fill some of the columns with 0 where it makes sense
to_be_filled = ["All Students - American Indian/Alaska Native",
               "All Students - Asian",
               "All Students - Black or African American",
               "All Students - Native Hawai'ian or Other Pacific Islander",
               "All Students - Hispanic",
               "All Students - Two or More Races",
               "All Students - White"]

# Use .loc to avoid SettingWithCopyWarning
regular_schools.loc[:, to_be_filled] = regular_schools[to_be_filled].fillna(0)


In [ ]:
# Verify changes to nulls and prepare to drop na
print(regular_schools.isnull().sum())


In [ ]:
# Drop remaining nulls wher it would make sense
regular_schools = regular_schools.dropna()


In [ ]:
# Verify changes to nulls and no nulls should remain
print(regular_schools.isnull().sum())


## Plotting to help Clean the Data

In [ ]:
# Plot a histogram of Total enrolled students and check for outliers
regular_schools["Total students all grades (includes AE)"].plot(kind='hist',bins=100)
regular_schools["Total students all grades (includes AE)"].nlargest(5)

In [ ]:
# Plot a histogram of Total Teachers and check for outliers
regular_schools["Total Teachers"].plot(kind='hist',bins=100)
regular_schools["Total Teachers"].nlargest(5)

In [ ]:
# Plot a histogram of Student Teacher Ratio and check for outliers
regular_schools["Student teacher ratio"].plot(kind='hist',bins=100)
regular_schools["Student teacher ratio"].nlargest(5)

In [ ]:
# Re-Plot a histogram of Total enrolled students after removing 0s and outliers above the 99th quantile
q_stu_high = regular_schools["Total students all grades (includes AE)"].quantile(0.99)
total_filtered = regular_schools[(regular_schools['Total students all grades (includes AE)'] > 0) & (regular_schools['Total students all grades (includes AE)'] <= q_stu_high)]

total_filtered["Total students all grades (includes AE)"].plot(kind='hist',bins=100)
total_filtered["Total students all grades (includes AE)"].nlargest(5)


In [ ]:
# Re-Plot a histogram of Total Teachers after removing 0s and outliers above the 99th quantile
q_teach_high = regular_schools["Total Teachers"].quantile(0.99)
fte_filtered = regular_schools[(regular_schools["Total Teachers"] > 0) & (regular_schools["Total Teachers"] <= q_teach_high)]

fte_filtered["Total Teachers"].plot(kind='hist',bins=100)
fte_filtered["Total Teachers"].nlargest(5)


In [ ]:
# Re-Plot a histogram of Student Teacher Ratio after removing 0s and outliers above the 99th quantile
q_ratio_high = regular_schools["Student teacher ratio"].quantile(0.99)
ratio_filtered = regular_schools[(regular_schools["Student teacher ratio"] > 0) & (regular_schools["Student teacher ratio"] <= q_ratio_high)]

ratio_filtered["Student teacher ratio"].plot(kind='hist',bins=100)
ratio_filtered["Student teacher ratio"].nlargest(5)

#Total df with all col filtered:
regular_filt_df = regular_schools[
    ((regular_schools["Student teacher ratio"] > 0) & (regular_schools["Student teacher ratio"] <= q_ratio_high)) & 
    ((regular_schools["Total Teachers"] > 0) &  (regular_schools["Total Teachers"] <= q_teach_high)) & 
    ((regular_schools['Total students all grades (includes AE)'] > 0) & 
     (regular_schools['Total students all grades (includes AE)'] <= q_stu_high))
]

## Correlation Plot to Understand Data Relationships

In [ ]:
# Use seaborn to generate a correlation heatmap for the columns in the dataframe
correlartion_df = regular_schools.drop(columns=["Location state", "School level", "School type (description)"]).corr()
correlation_df

sns.heatmap(correlation_df, vmin=-1, vmax=1)


## Geographic Plots

In [ ]:
# Plot only schools from the 48 contiguous US states or the 50 US states (drop territories and PR)
print(regular_schools["Location state"].nunique())
print(regular_schools["Location state"].unique())

contiguous_df = regular_schools.loc[~regular_schools["Location state"].isin(["PR","VI","AK","HI","GU","DC"])]

contiguous_df.plot.scatter(x="Longitude", y="Latitude", figsize=(15,9))
# NOTE: Why does IL not report this data? Or Utah?
# NOTE: There is ONE school reporting within UT, Aneth Community School, which is in Navajo Nation

In [ ]:
# Plot only schools in your state
pa_df = regular_schools.loc[regular_schools["Location state"] == "PA"]
pa_df.plot.scatter(x="Longitude",y="Latitude",figsize=(15,9))




## School Quality Analysis: Student/Teacher Ratios

In [ ]:
# Use the dataframe that has filtered out the outliers in Student Teacher Ratio and group by state and get the average
# Show the highest 5 Student/Teacher ratios and lowest 5 Student/Teacher ratios as well as the average for your state
grouped_s2t = ratio_filtered["Student teacher ratio"].groupby(ratio_filtered["Location state"]).mean().sort_values(ascending=False)
s2t = round(grouped_s2t,2)

print("-- Student/Teacher Ratios --")
# Display the top 5 and bottom 5 values
print("5 Highest :")
print(s2t.head(5))
print("\n5 Lowest:")
print(s2t.tail(5))

print(f"\nPennsylvania STR: {s2t.PA}")



In [ ]:
# What is the mean of the Student/Teacher Ratio
print(f"The mean Student/Teacher Ratio weighted equally by state is: {round(s2t.mean(),2)}.")
print(f"The mean ratio across all schools is {round(ratio_filtered['Student teacher ratio'].mean(),2)}.")



In [ ]:
# What is the standard deviation of the Student/Teacher Ratio
print(f"State level standard deviaton of STR: {round(s2t.std(),2)}")


In [ ]:
# Plot the states ratios in descending order
s2t.plot.bar(x="Location state", y="Student teacher ratio", figsize = (12,4), rot=70)


In [ ]:
# Plot a boxplot of the Student/Teacher Ratios
s2t.plot.box(title = "National Student Teacher Ratio")


## Summarize your findings from any of the analysis or plots above:
* Were there any interesting Correlations?
    * It was interesting that there was virtually no correlation between student teacher ratio and racial demographic.
    * There was a negative correlation with Latitude and number of hispanic students, as one might expect.
    * There was a positive correlation between number of white students and number of mixed race students, but the correlation did not extend to other races besides white, except maybe VERY weakly in asian.
* How did your state compare to others?
    * PA (13.77) had a lower than national average (14.17) student to teacher ratio, within one stdev (2.26).
* Would you have made any different decisions in the earlier cleaning stages after becoming more familiar with the data?
    * Kept in other school types to compare
* What is going on in IL?
    * It would appear that the Department of Education is not publishing these data, as I cannot find any information about Illinois not reporting it to Dept Ed.
    * Why would they do this?
    * I also note that there is only one (Navajo Nation) school reporting in Utah.
* What other analysis could be done with this data?
    * Compare Student Teacher Ratios or demographic data by school type across states
    * Observe number of students by race in each type of school
    * Racial demographics of students by state or by location (Lat/Long)

## BONUS: Generate other interesting analysis or plots

In [ ]:
# Racial demographics of students by state or by location (Lat/Long)
import matplotlib.pyplot as plt

contiguous_df2 = total_filtered.loc[~total_filtered["Location state"].isin(["PR","VI","AK","HI","GU","DC"])]

contiguous_df2.plot.scatter(x='Longitude', y='Latitude', s=contiguous_df2['All Students - American Indian/Alaska Native'])

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Density of Native American Students by Location')

plt.show()



In [ ]:
contiguous_df2.plot.scatter(x='Longitude', y='Latitude', s=contiguous_df2['All Students - Asian'])

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Density of Asian Students by Location')

plt.show()


In [ ]:
contiguous_df2.plot.scatter(x='Longitude', y='Latitude', s=contiguous_df2['All Students - Black or African American'])

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Density of Black Students by Location')

plt.show()


In [ ]:
contiguous_df2.plot.scatter(x='Longitude', y='Latitude', s=contiguous_df2['All Students - White'])

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Density of White Students by Location')

plt.show()


In [ ]:
contiguous_df2.plot.scatter(x='Longitude', y='Latitude', s=contiguous_df2['All Students - Hispanic'])

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Density of Hispanic Students by Location')

plt.show()
